<a href="https://colab.research.google.com/github/DavinciB/child_grooming_detector/blob/main/src/SVM/Data%20Prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import xml.etree.ElementTree as ET #for parsing the xml file to a tree
import datetime
#store the path to the folder containing the training data
train_data_path = r"/content/drive/MyDrive/online-grooming-detector-master/data/pan12-sexual-predator-identification-training-corpus-2012-05-01"
#parse the training data xml file
training_xml = ET.parse(train_data_path + '/pan12-sexual-predator-identification-training-corpus-2012-05-01.xml')
#obtain the root of the tree we created above
root = training_xml.getroot()        


In [ ]:
#Adding index attribute to conversations and changing line attributes of messages for ease of pre processing
i = 0
for conversation in root:
  conversation.set('index',i)   #adding an index value to each conversation in the tree
  i += 1
for conversation in root:
  i = 0
  for message in conversation:
    message.set('line',i) #changing the line value of each messsage in all conversation to be in ordered form
    i += 1

In [ ]:
#Before removing single author conversations
print(root[1][0][2].text)

asuu


In [ ]:
#single author 
conv_2_remove = [] #list to store the conversation id of conversations to be removed
authors = [] #list of unique authors in each conversations
init_len = len(root)

for conversation in root:
    authors.clear()
    for message in conversation:
        author = message.find('author').text
        if author not in authors:
            authors.append(author)

    if (len(authors)) <= 1 and conversation.get('id') not in conv_2_remove:
        conv_2_remove.append(conversation.get('id'))

print(str(len(conv_2_remove))+" out of "+str(init_len)+" conversations contain single user")
sing_user = len(conv_2_remove)

12773 out of 66927 conversations contain single user


In [ ]:
#Before removing low message count conversations
print(root[2][0][2].text)

hi


In [ ]:
#Each user having 5 or less messages
for conversation in root:
    if conversation.get('id') in conv_2_remove:
        continue
    authors = {}
    for message in conversation:
        author = message.find('author').text
        if author in authors:
            authors[author] = authors[author] + 1
        else:
            authors[author] = 1
    remove = True
    for author in authors:
        if authors[author] > 5:
            remove = False
    if remove is True and conversation.get('id') not in conv_2_remove:
        conv_2_remove.append(conversation.get('id'))
low_msg_count = len(conv_2_remove) - sing_user
print(str(low_msg_count)+" out of "+str(init_len)+" conversations have low message count")

39054 out of 66927 conversations have low message count


In [ ]:
for conversation in root.findall('conversation'):
    if conversation.get('id') in conv_2_remove:
        root.remove(conversation)
print("The new root has a length of " + str(len(root)))
print(str(init_len - len(root))+" conversations removed")

The new root has a length of15100
51827 conversations removed


In [ ]:
#Adding index attribute to conversations and changing line attributes of messages
i = 0
for conversation in root:
  conversation.set('index',i)   
  i += 1
for conversation in root:
  i = 0
  for message in conversation:
    message.set('line',i)
    i += 1

In [ ]:
#After removing single author conversations
print(root[1][0][2].text)

happy is ayuppie word.


In [ ]:
#Afetr removing low message count conversations
print(root[2][0][2].text)

hi


In [ ]:
#Removing sequences of meaningless characters
import re
messages = []
count = 0
for conversation in root:
  messages.clear()
  conv = conversation.get('id')
  index = conversation.get('index')
  for message in conversation:
    text = message.find("text").text
    if text is None or len(text) < 20:
      continue
    match_str = re.findall("[\W_]", text)
    if len(match_str) / len(text) > 0.6:
      line = message.get('line')
      messages.append(int(line))
    for i in reversed(messages):
      try:
        root[int(index)].remove(root[int(index)][i])
        count += 1
      except:
        continue
print(str(count)+" messages has been removed")

4484 messages has been removed


In [ ]:
#Removing index and updating line attributes to str else it gives errror
#during converting to xml
for i in range(len(root)):
  root[i].attrib.pop('index') 
for conversation in root:
  i = 1
  for message in conversation:
    message.set('line',str(i))
    i += 1

In [ ]:
from xml.etree.ElementTree import ElementTree,tostring
tree = ElementTree(root)
tree.write(open(r'/content/drive/MyDrive/online-grooming-detector-master/data/svm_training_data/training_data.xml', 'wb'))
print("Filtered data written!")


Filtered data written!
